In [109]:
import mysql.connector

# Establish a connection to the MySQL database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="github"
)

# Create a cursor
cursor = conn.cursor()

# SQL code for creating tables
sql_code = """
DROP TABLE IF EXISTS RepoTopic;
DROP TABLE IF EXISTS RepoContributor;
DROP TABLE IF EXISTS RepoLanguage;
DROP TABLE IF EXISTS Language;
DROP TABLE IF EXISTS Topic;
DROP TABLE IF EXISTS Contributor;
DROP TABLE IF EXISTS Repo;

CREATE TABLE Repo (
    id INT PRIMARY KEY AUTO_INCREMENT,
    Name VARCHAR(255),
    Description LONGTEXT,
    Stars INT,
    URL VARCHAR(255),
    DateCreation DATE,
    LastModification DATE,
    Forks INT
);

CREATE TABLE Contributor (
    id INT PRIMARY KEY AUTO_INCREMENT,
    login VARCHAR(255)
);

CREATE TABLE Topic (
    id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(255)
);

CREATE TABLE Language (
    id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(255)
);

CREATE TABLE RepoLanguage (
    repo_id INT,
    language_id INT,
    FOREIGN KEY (repo_id) REFERENCES Repo(id),
    FOREIGN KEY (language_id) REFERENCES Language(id),
    is_main BOOLEAN
);

CREATE TABLE RepoContributor (
    repo_id INT,
    contributor_id INT,
    contributions INT,
    FOREIGN KEY (repo_id) REFERENCES Repo(id),
    FOREIGN KEY (contributor_id) REFERENCES Contributor(id)
);

CREATE TABLE RepoTopic (
    topic_id INT,
    repo_id INT,
    FOREIGN KEY (topic_id) REFERENCES Topic(id),
    FOREIGN KEY (repo_id) REFERENCES Repo(id)
);
"""

# Execute the SQL code
for statement in sql_code.split(";"):
    if statement.strip() != "":
        cursor.execute(statement)

# Commit the changes to the database
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

print('done')

done


In [110]:
import pandas as pd
import ast
df = pd.read_csv('merged_file.csv')
df = df.drop_duplicates()
df = df.dropna(subset=['Language'])
df['contributors'] = df['contributors'].replace("['The history or contributor list is too large to list contributors for this repository via the API.']", "[]")
df['Description'] = df['Description'].fillna("has no description")
df['Topics'] = df['Topics'].apply(ast.literal_eval)
df['languages'] = df['languages'].apply(ast.literal_eval)
df['contributors'] = df['contributors'].apply(ast.literal_eval)
df

,Name,Description,Stars,URL,Date creation,Dernier modification,Forks,Language,Topics,languages,contributors
0,freeCodeCamp,freeCodeCamp.org's open-source codebase and cu...,369549,https://github.com/freeCodeCamp/freeCodeCamp,2014-12-24T17:49:19Z,2023-07-06T10:34:34Z,32806,TypeScript,"[careers, certification, community, curriculum...","[TypeScript, JavaScript, CSS, MDX, Dockerfile,...","[{'login': 'raisedadead', 'contributions': 142..."
5,public-apis,A collective list of free APIs,247051,https://github.com/public-apis/public-apis,2016-03-20T23:49:42Z,2023-07-06T10:46:14Z,28345,Python,"[api, apis, dataset, development, free, list, ...","[Python, Shell]","[{'login': 'matheusfelipeog', 'contributions':..."
6,developer-roadmap,"Interactive roadmaps, guides and other educati...",243636,https://github.com/kamranahmedse/developer-roa...,2017-03-15T13:45:52Z,2023-07-06T10:42:38Z,34240,TypeScript,"[angular-roadmap, backend-roadmap, blockchain-...","[TypeScript, Astro, JavaScript, CSS]","[{'login': 'kamranahmedse', 'contributions': 1..."
7,system-design-primer,Learn how to design large-scale systems. Prep ...,223929,https://github.com/donnemartin/system-design-p...,2017-02-26T16:15:28Z,2023-07-06T10:35:11Z,39662,Python,"[design, design-patterns, design-system, devel...","[Python, Shell]","[{'login': 'donnemartin', 'contributions': 175..."
8,react,The library for web and native user interfaces,210013,https://github.com/facebook/react,2013-05-24T16:15:54Z,2023-07-06T10:36:32Z,43969,JavaScript,"[declarative, frontend, javascript, library, r...","[JavaScript, HTML, CSS, C++, TypeScript, Coffe...","[{'login': 'zpao', 'contributions': 1778, 'typ..."
...,...,...,...,...,...,...,...,...,...,...,...
2698,editor.js,A block-style editor with clean JSON output,23467,https://github.com/codex-team/editor.js,2015-11-28T20:56:35Z,2023-07-06T10:04:56Z,1791,TypeScript,"[codex-editor, editor, javascript, json, redac...","[TypeScript, HTML, CSS, JavaScript]","[{'login': 'khaydarov', 'contributions': 202, ..."
2699,Hystrix,Hystrix is a latency and fault tolerance libra...,23465,https://github.com/Netflix/Hystrix,2012-11-19T20:14:46Z,2023-07-06T06:42:21Z,4710,Java,[],"[Java, Clojure, CSS, Groovy]","[{'login': 'mattrjacobs', 'contributions': 538..."
2700,awesome-swift,A collaborative list of awesome Swift librarie...,23463,https://github.com/matteocrippa/awesome-swift,2014-07-10T14:04:09Z,2023-07-06T06:29:10Z,3051,Ruby,"[awesome, awesome-list, awesome-lists, awesome...",[Ruby],"[{'login': 'READMEbot', 'contributions': 733, ..."
2701,pkg,Package your Node.js project into an executable,23454,https://github.com/vercel/pkg,2016-08-08T19:41:59Z,2023-07-06T10:29:54Z,1005,JavaScript,"[binaries, cli, compiler, nodejs]","[JavaScript, TypeScript, CSS, HTML, Pug, Coffe...","[{'login': 'igorklopov', 'contributions': 887,..."


In [111]:
import mysql.connector
from datetime import datetime

# Establish a connection to the MySQL database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="github"
)

# Create a cursor
cursor = conn.cursor()

# Insert data from DataFrame
for _, row in df.iterrows():
    # Insert Repo data
    repo_data = {
        "Name": row['Name'],
        "Description": row['Description'],
        "Stars": row['Stars'],
        "URL": row['URL'],
        "DateCreation": datetime.strptime(row['Date creation'], '%Y-%m-%dT%H:%M:%SZ').date(),
        "LastModification": datetime.strptime(row['Dernier modification'], '%Y-%m-%dT%H:%M:%SZ').date(),
        "Forks": row['Forks']
    }

    repo_insert_query = """
    INSERT INTO Repo (Name, Description, Stars, URL, DateCreation, LastModification, Forks)
    VALUES (%(Name)s, %(Description)s, %(Stars)s, %(URL)s, %(DateCreation)s, %(LastModification)s, %(Forks)s)
    """
    cursor.execute(repo_insert_query, repo_data)

    repo_id = cursor.lastrowid
    
    # *---- Start Insert Languages ----*
    languages = row['languages']
    language_ids = {}

    language_insert_query = """
    INSERT INTO Language (name)
    VALUES (%s)
    """
    for language in languages:
        language_id_query = "SELECT id FROM Language WHERE name = %s"
        cursor.execute(language_id_query, (language,))
        result = cursor.fetchone()
        if result:
            language_id = result[0]
        else:
            is_main = (language == row['Language'])
            cursor.execute(language_insert_query, (language,))
            language_id = cursor.lastrowid
        language_ids[language] = language_id
        
            # Insert RepoLanguage
    repo_language_insert_query = """
    INSERT INTO RepoLanguage (repo_id, language_id)
    VALUES (%s, %s)
    """

    for language in language_ids:
        cursor.execute(repo_language_insert_query, (repo_id, language_ids[language]))
    
    # !---- End Insert Languages ----!
    
    # *---- Start Insert Topics ----*
    topics = row['Topics']
    topic_ids = {}

    topic_insert_query = """
    INSERT INTO Topic (name)
    VALUES (%s)
    """
    for topic in topics:
        topic_id_query = "SELECT id FROM Topic WHERE name = %s"
        cursor.execute(topic_id_query, (topic,))
        result = cursor.fetchone()
        if result:
            topic_id = result[0]
        else:
            cursor.execute(topic_insert_query, (topic,))
            topic_id = cursor.lastrowid
        topic_ids[topic] = topic_id

            # Insert RepoTopic
    repo_topic_insert_query = """
    INSERT INTO RepoTopic (repo_id, topic_id)
    VALUES (%s, %s)
    """

    for topic in topic_ids:
        cursor.execute(repo_topic_insert_query, (repo_id, topic_ids[topic]))

    # !---- End Insert Topics ----!

    # *---- Start Insert Contributors ----*
    contributors = row['contributors']
    contributor_ids = {}

    contributor_insert_query = """
    INSERT INTO Contributor (login)
    VALUES (%s)
    """

    for contributor in contributors:
        login = contributor['login']
        contributions = contributor['contributions']
        contributor_id_query = "SELECT id FROM Contributor WHERE login = %s"
        cursor.execute(contributor_id_query, (login,))
        result = cursor.fetchone()
        if result:
            contributor_id = result[0]
        else:
            cursor.execute(contributor_insert_query, (login,))
            contributor_id = cursor.lastrowid
        contributor_ids[login] = {"contributor_id":contributor_id,"contributions":contributions}



    # Insert RepoContributor
    repo_contributor_insert_query = """
    INSERT INTO RepoContributor (repo_id, contributor_id, contributions)
    VALUES (%s, %s, %s)
    """
    for contributor in contributor_ids:
        cursor.execute(repo_contributor_insert_query, (repo_id, contributor_ids[contributor]['contributor_id'], contributor_ids[contributor]['contributions']))

    # ---- End Insert Contributors ---!

    print("_", _)

# Commit the changes to the database
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()


_ 0
_ 5
_ 6
_ 7
_ 8
_ 10
_ 11
_ 12
_ 13
_ 16
_ 17
_ 18
_ 19
_ 20
_ 22
_ 24
_ 25
_ 27
_ 28
_ 29
_ 30
_ 31
_ 32
_ 33
_ 34
_ 36
_ 37
_ 38
_ 39
_ 40
_ 43
_ 44
_ 45
_ 46
_ 47
_ 48
_ 49
_ 50
_ 51
_ 52
_ 53
_ 54
_ 55
_ 56
_ 57
_ 58
_ 59
_ 60
_ 61
_ 62
_ 63
_ 64
_ 65
_ 66
_ 67
_ 68
_ 69
_ 70
_ 71
_ 72
_ 73
_ 75
_ 76
_ 77
_ 78
_ 79
_ 80
_ 81
_ 82
_ 83
_ 84
_ 85
_ 86
_ 88
_ 89
_ 90
_ 91
_ 92
_ 93
_ 94
_ 96
_ 97
_ 98
_ 100
_ 101
_ 102
_ 103
_ 105
_ 106
_ 107
_ 108
_ 109
_ 110
_ 111
_ 112
_ 114
_ 115
_ 116
_ 117
_ 118
_ 120
_ 122
_ 123
_ 124
_ 125
_ 126
_ 127
_ 128
_ 129
_ 130
_ 131
_ 132
_ 134
_ 135
_ 136
_ 137
_ 138
_ 139
_ 140
_ 142
_ 143
_ 144
_ 145
_ 146
_ 147
_ 148
_ 149
_ 150
_ 151
_ 153
_ 154
_ 155
_ 156
_ 157
_ 158
_ 160
_ 161
_ 162
_ 163
_ 164
_ 165
_ 166
_ 167
_ 168
_ 169
_ 170
_ 171
_ 172
_ 173
_ 174
_ 175
_ 176
_ 177
_ 178
_ 180
_ 181
_ 182
_ 183
_ 184
_ 185
_ 186
_ 187
_ 189
_ 190
_ 191
_ 192
_ 194
_ 195
_ 196
_ 197
_ 198
_ 199
_ 201
_ 202
_ 203
_ 204
_ 205
_ 206
_ 207
_ 208
_ 209
_ 

_ 1577
_ 1578
_ 1579
_ 1580
_ 1581
_ 1582
_ 1583
_ 1584
_ 1585
_ 1586
_ 1587
_ 1588
_ 1589
_ 1590
_ 1591
_ 1592
_ 1593
_ 1594
_ 1595
_ 1596
_ 1597
_ 1598
_ 1599
_ 1601
_ 1602
_ 1603
_ 1604
_ 1605
_ 1606
_ 1607
_ 1608
_ 1609
_ 1610
_ 1611
_ 1612
_ 1613
_ 1614
_ 1616
_ 1617
_ 1618
_ 1619
_ 1620
_ 1621
_ 1622
_ 1623
_ 1624
_ 1625
_ 1627
_ 1628
_ 1629
_ 1630
_ 1631
_ 1632
_ 1634
_ 1635
_ 1636
_ 1637
_ 1639
_ 1640
_ 1641
_ 1642
_ 1643
_ 1645
_ 1646
_ 1649
_ 1650
_ 1651
_ 1653
_ 1654
_ 1655
_ 1656
_ 1657
_ 1658
_ 1659
_ 1660
_ 1661
_ 1663
_ 1664
_ 1665
_ 1666
_ 1667
_ 1668
_ 1669
_ 1670
_ 1671
_ 1672
_ 1673
_ 1674
_ 1675
_ 1676
_ 1677
_ 1678
_ 1679
_ 1680
_ 1681
_ 1682
_ 1683
_ 1684
_ 1685
_ 1686
_ 1687
_ 1688
_ 1690
_ 1692
_ 1693
_ 1694
_ 1695
_ 1696
_ 1697
_ 1699
_ 1702
_ 1703
_ 1705
_ 1706
_ 1707
_ 1708
_ 1709
_ 1710
_ 1711
_ 1714
_ 1715
_ 1716
_ 1717
_ 1719
_ 1720
_ 1721
_ 1722
_ 1723
_ 1724
_ 1725
_ 1726
_ 1727
_ 1728
_ 1729
_ 1730
_ 1731
_ 1732
_ 1733
_ 1735
_ 1736
_ 1737
_ 1738
_ 1740